In [ ]:
#Imports
import numpy as np
import pandas as pd
from scipy import linalg
from scipy.interpolate import make_interp_spline
import pdb
import sys
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches
import pickle

sys.path.append('path to the folder containing PRASTOF.py')
from PRASTOF import Prastof

# Solve

In [ ]:
data_file = 'SI1_PRASTOF_data.xlsx'
dMFA = Prastof.from_template(data_file)

In [ ]:
dMFA.solve()

In [ ]:
inflow = dMFA.inflow 
inflow[2015] = inflow[2016]
outflow = dMFA.outflow 
stock = dMFA.stock
stock_cap = dMFA.stock_cap.loc(axis=1)[2015:2100]

# Function to plot evolution of 5 regions - 5 SSP

In [ ]:
def subplot_SSP_region(df, y_label, suptitle, y_ticks):
    #SSP and region list
    df_SSP  = list(dict.fromkeys(df.index.get_level_values(0)))
    df_reg  = list(dict.fromkeys(df.index.get_level_values(1)))

    fig = plt.figure(figsize=(12,8))
    fig.subplots_adjust(hspace=0.05, wspace=0.05)
    
    #For all regions
    for i in np.arange(0, len(df_reg)):
        ax = fig.add_subplot(2, 3, i+1)

        #For all SSP
        colors = ['green','blue','red','orange','purple']
        #colors = colors5
        for j in np.arange(0, len(df_SSP)): 
            x = df.columns
            y = df.loc(axis=0)[:,df_reg[i]].droplevel(level=1).loc[df_SSP[j]]
            ax.plot(x, y, c=colors[j], lw=3)
            ax.yaxis.set_minor_locator(plt.MultipleLocator(y_ticks))
            
        #If 5 subplot
        if len(df_reg) == 5:
            #Remove x axis of 2 first
            if i == 0 or i == 1:
                ax.xaxis.set_ticklabels('')
                ax.xaxis.set_minor_locator(plt.MultipleLocator(5))
            #Remove y axis of 3 rights
            if i == 1 or i == 2 or i == 4:
                ax.xaxis.set_minor_locator(plt.MultipleLocator(5))
                ax.yaxis.set_ticklabels('')
        
        #If 6 subplots
        if len(df_reg) == 6:
            for ax in fig.get_axes():
                ax.label_outer()
                ax.xaxis.set_minor_locator(plt.MultipleLocator(5))

        #Add y label
        if i == 0 or i == 3:
            plt.ylabel(y_label)

        #Start axis by 0
        ax.set_ylim(ymin=0, ymax= df.max().max()*1.08)
        
        #Grid lines
        plt.grid(linestyle = '--', linewidth = 0.5, alpha = 0.75)

        # Title of subplt
        plt.rcParams['axes.titlepad'] = -16
        
        plt.title(df_reg[i], size=15, loc='center')

    #Add legend
    
    if len(df_reg) == 5:
        plt.legend(df_SSP, fontsize=20, loc=(1.25, 0.10))
    if len(df_reg) == 6:
        plt.legend(df_SSP, fontsize=20, loc=(1.105, 0.7))
        
    #Add suptitle
    plt.grid(linestyle = '--', linewidth = 0.5, alpha = 0.75)
    fig.suptitle(suptitle, fontsize=25)
    
    

# Figure 3 - Stock per capita

In [ ]:
df = stock_cap.loc[:,:,'Medium'].sum(level=[0,1])
subplot_SSP_region(df,'[kg/cap]','', 25)
#print(df.loc(axis=1)[2015,2020, 2030,2040, 2050,2060,2070, 2080,2090, 2100])

plt.savefig('figure3.pdf', dpi = 900, bbox_inches='tight')

#  Figure 4  - Stock / Inflow / Outflow - Contribution and SSP

In [ ]:
#Create df with inflow, outflow and stock according a specific SSP and a specific level
ssp = 'SSP2'
level = 'Medium'
inf_ssp = inflow.loc[:,:,level].sum(level=[0]).loc(axis=1)[2015,2020,2025,2030,2035,2040,2045,2050,2055,2060,2065,2070,2075,2080,2085,2090,2095,2100]
inf_cont_SSP2 = inflow.loc[ssp,:,level].sum(level=[1,2,3])

out_ssp = outflow.loc[:,:,level].sum(level=[0]).loc(axis=1)[2015,2020,2025,2030,2035,2040,2045,2050,2055,2060,2065,2070,2075,2080,2085,2090,2095,2100]
out_cont_SSP2 = outflow.loc[ssp,:,level].sum(level=[1,2,3])

stock_ssp = stock.loc[:,:,level].sum(level=[0]).loc(axis=1)[2015,2020,2025,2030,2035,2040,2045,2050,2055,2060,2065,2070,2075,2080,2085,2090,2095,2100]
stock_cont_SSP2 = stock.loc[ssp,:,level].sum(level=[1,2,3])

In [ ]:
x = stock.columns

fig, (ax1, ax2, ax3) = plt.subplots(3)
fig.set_figheight(12)
fig.set_figwidth(12)
fig.subplots_adjust(hspace=0.1)

colors = ['#67001f','#b2182b', '#d6604d','#f4a582','#fddbc7','#f7f7f7','#d1e5f0','#92c5de','#4393c3','#2166ac','#053061']

# ax1 - Stock
#Stacked bar for sector
BC_res = stock_cont_SSP2.loc['BC'].loc['BC'].loc['Residential']
BC_com = stock_cont_SSP2.loc['BC'].loc['BC'].loc['Commercial']
Trans_auto = stock_cont_SSP2.loc['Trans'].loc['Auto'].sum()
Trans_fr = stock_cont_SSP2.loc['Trans'].loc['Freight'].sum()
Trans_oth = stock_cont_SSP2.loc['Trans'].loc['Trans'].sum()
EE_dist = stock_cont_SSP2.loc['EE'].loc['Distribution'].sum()
EE_gen = stock_cont_SSP2.loc['EE'].loc['Generation'].sum()
ME = stock_cont_SSP2.loc['ME'].sum()
CD = stock_cont_SSP2.loc['CD'].sum()
PC_cans = stock_cont_SSP2.loc['PC'].loc['Cans'].sum()
PC_oth = stock_cont_SSP2.loc['PC'].loc['Other'].sum()

x = stock_cont_SSP2.columns
width = 5
labels = ['BC residential', 'BC comercial', 'Transport automotive', 'Trans freight', 'Transport other',
'EE distreibtuion', 'EE geneneration', 'ME', 'CD', 'PC cans', 'PC other']
ax1.stackplot(x,  BC_res, BC_com,  Trans_auto, Trans_fr, Trans_oth, EE_dist, EE_gen, ME, CD, PC_cans, PC_oth, 
             colors= colors, alpha = 1, labels= labels, )#edgecolor = 'k')

#SSP evolution
y_ssp1 = stock_ssp.loc['SSP1']
y_ssp2 = stock_ssp.loc['SSP2']
y_ssp3 = stock_ssp.loc['SSP3']
y_ssp4 = stock_ssp.loc['SSP4']
y_ssp5 = stock_ssp.loc['SSP5']

ax1.plot(y_ssp1, color='k', linewidth=3, ls=':', marker='s', markersize=6)
ax1.plot(y_ssp2, color='k', linewidth=4)
ax1.plot(y_ssp3, color='k', linewidth=3, ls=':', marker='d', markersize=6)
ax1.plot(y_ssp4, color='k', linewidth=3, ls=':', marker='*', markersize=8)
ax1.plot(y_ssp5, color='k', linewidth=3, ls=':', marker='h', markersize=7)
ax1.set_ylim(ymin=0, ymax=3500)
ax1.yaxis.set_minor_locator(plt.MultipleLocator(125))
ax1.set(ylabel='Stock [Mt]')


# ax2 - Plot inflow

#Stacked bar for sector
BC_res = inf_cont_SSP2.loc['BC'].loc['BC'].loc['Residential']
BC_com = inf_cont_SSP2.loc['BC'].loc['BC'].loc['Commercial']
Trans_auto = inf_cont_SSP2.loc['Trans'].loc['Auto'].sum()
Trans_fr = inf_cont_SSP2.loc['Trans'].loc['Freight'].sum()
Trans_oth = inf_cont_SSP2.loc['Trans'].loc['Trans'].sum()
EE_dist = inf_cont_SSP2.loc['EE'].loc['Distribution'].sum()
EE_gen = inf_cont_SSP2.loc['EE'].loc['Generation'].sum()
ME = inf_cont_SSP2.loc['ME'].sum()
CD = inf_cont_SSP2.loc['CD'].sum()
PC_cans = inf_cont_SSP2.loc['PC'].loc['Cans'].sum()
PC_oth = inf_cont_SSP2.loc['PC'].loc['Other'].sum()

x = inf_cont_SSP2.columns
width = 5
labels = ['BC residential', 'BC comercial', 'Transport automotive', 'Trans freight', 'Transport other',
'EE distreibtuion', 'EE geneneration', 'ME', 'CD', 'PC cans', 'PC other']
ax2.stackplot(x,  BC_res, BC_com,  Trans_auto, Trans_fr, Trans_oth, EE_dist, EE_gen, ME, CD, PC_cans, PC_oth, 
             colors= colors, alpha = 1, labels= labels,) #edgecolor = 'k')

#SSP evolution
y_ssp1 = inf_ssp.loc['SSP1']
y_ssp2 = inf_ssp.loc['SSP2']
y_ssp3 = inf_ssp.loc['SSP3']
y_ssp4 = inf_ssp.loc['SSP4']
y_ssp5 = inf_ssp.loc['SSP5']

ax2.plot(y_ssp1, color='k', linewidth=3, ls=':', marker='s', markersize=6)
ax2.plot(y_ssp2, color='k', linewidth=4)
ax2.plot(y_ssp3, color='k', linewidth=3, ls=':', marker='d', markersize=6)
ax2.plot(y_ssp4, color='k', linewidth=3, ls=':', marker='*', markersize=8)
ax2.plot(y_ssp5, color='k', linewidth=3, ls=':', marker='h', markersize=7)

ax2.set_ylim(ymin=0, ymax=175)
ax2.yaxis.set_minor_locator(plt.MultipleLocator(5))
ax2.set(ylabel='Inflow [Mt]')

 
# ax3 - Plot outflow

#Stacked bar for sector
BC_res = out_cont_SSP2.loc['BC'].loc['BC'].loc['Residential']
BC_com = out_cont_SSP2.loc['BC'].loc['BC'].loc['Commercial']
Trans_auto = out_cont_SSP2.loc['Trans'].loc['Auto'].sum()
Trans_fr = out_cont_SSP2.loc['Trans'].loc['Freight'].sum()
Trans_oth = out_cont_SSP2.loc['Trans'].loc['Trans'].sum()
EE_dist = out_cont_SSP2.loc['EE'].loc['Distribution'].sum()
EE_gen = out_cont_SSP2.loc['EE'].loc['Generation'].sum()
ME = out_cont_SSP2.loc['ME'].sum()
CD = out_cont_SSP2.loc['CD'].sum()
PC_cans = out_cont_SSP2.loc['PC'].loc['Cans'].sum()
PC_oth = out_cont_SSP2.loc['PC'].loc['Other'].sum()

x = inf_cont_SSP2.columns
width = 5
labels = ['BC residential', 'BC comercial', 'Trans automotive', 'Trans freight', 'Trans other',
'EE distribution', 'EE generation', 'ME', 'CD', 'PC cans', 'PC other']
ax3.stackplot(x,  BC_res, BC_com,  Trans_auto, Trans_fr, Trans_oth, EE_dist, EE_gen, ME, CD, PC_cans, PC_oth, 
             colors= colors, alpha = 1, labels= labels,)# edgecolor = 'white')

#SSP evolution
y_ssp1 = out_ssp.loc['SSP1']
y_ssp2 = out_ssp.loc['SSP2']
y_ssp3 = out_ssp.loc['SSP3']
y_ssp4 = out_ssp.loc['SSP4']
y_ssp5 = out_ssp.loc['SSP5']

ax3.plot(y_ssp1, color='k', linewidth=3, ls=':', marker='s', markersize=6)
ax3.plot(y_ssp2, color='k', linewidth=4)
ax3.plot(y_ssp3, color='k', linewidth=3, ls=':', marker='d', markersize=6)
ax3.plot(y_ssp4, color='k', linewidth=3, ls=':', marker='*', markersize=8)
ax3.plot(y_ssp5, color='k', linewidth=3, ls=':', marker='h', markersize=7)

ax3.set_ylim(ymin=0, ymax=175)
ax3.yaxis.set_minor_locator(plt.MultipleLocator(5))
ax3.set(ylabel='Outflow [Mt]')
    
#Remove axis
for ax in fig.get_axes():
    ax.label_outer()
    ax.xaxis.set_minor_locator(plt.MultipleLocator(5))
    #Start y axis by 0
    ax.set_ylim(ymin=0)
    
#Legends
legend_stages = plt.legend(reversed(plt.legend().legendHandles), reversed(labels), title = 'Sectors', loc=(1.02, 0.8))
legend = plt.gca().add_artist(legend_stages)
    
#Add legend
plt.legend(['SSP1', 'SSP2', 'SSP3','SSP4','SSP5'], title='Scenarios', loc=((1.065, 1.8)))
ax.xaxis.set_minor_locator(plt.MultipleLocator(5))


# x label
plt.xlabel('year')
ax.xaxis.set_minor_locator(plt.MultipleLocator(5))
plt.xticks(list(range(2020,2101,10)))


#Remove axis
for ax in fig.get_axes():
    ax.label_outer()
    #Start y axis by 0
    ax.set_ylim(ymin=0)
    ax.set_xlim(xmin = 2013 , xmax=2102)
    
plt.savefig('figure4.pdf', dpi = 900, bbox_inches='tight')

# Figure 5 alloys and alloying elements

In [ ]:
ssp = 'SSP2'
level = 'Medium'

stock_alloys = stock.loc[ssp,:,level].sum(level=4)
inf_alloys = inflow.loc[ssp,:,level].sum(level=4)
out_alloys = outflow.loc[ssp,:,level].sum(level=4)

In [ ]:
x = stock.columns


fig, (ax1, ax2, ax3) = plt.subplots(3)
fig.set_figwidth(12)
fig.set_figheight(8)

fig.subplots_adjust(hspace=0.1)

colors = ['#67001f','#b2182b', '#d6604d','#f4a582','#fddbc7','#f7f7f7','#d1e5f0','#92c5de','#4393c3','#2166ac','#053061']

# ax1 - Stock
#Stacked bar per alloys

x = inf_alloys.columns

s1000 = stock_alloys.loc['1000 series']
s2000 = stock_alloys.loc['2000 series']
a3004 = stock_alloys.loc[3004]
o3000 = stock_alloys.loc[3003] + stock_alloys.loc['other 3000 series']
a5182 = stock_alloys.loc[5182]
o5000 = stock_alloys.loc[5052]  + stock_alloys.loc['other 5000 series']
a6063 = stock_alloys.loc[6063]
a6000 = stock_alloys.loc['Other 6000 series'] + stock_alloys.loc[6061]
s7000 = stock_alloys.loc['7000 series'] 
s8000 = stock_alloys.loc['8000 series'] 
cast_al = stock_alloys.loc['Cast'] + stock_alloys.loc['Die-Cast']

labels = ['1000 series','2000 series','3004 alloy','Other 3000 series','5182 alloys','Other 5000 series','6063 alloy','6000 series','7000 series','8000 series','Cast alloys']
ax1.stackplot(x,  s1000, s2000, a3004, o3000, a5182, o5000, a6063, a6000, s7000, s8000, cast_al, colors = colors, labels=labels)
ax1.yaxis.set_minor_locator(plt.MultipleLocator(125))
ax1.set(ylabel='Stock [Mt]')

# ax2 - inflow
#Stacked bar per alloys

x = inf_alloys.columns

s1000 = inf_alloys.loc['1000 series']
s2000 = inf_alloys.loc['2000 series']
a3004 = inf_alloys.loc[3004]
o3000 = inf_alloys.loc[3003] + inf_alloys.loc['other 3000 series']
a5182 = inf_alloys.loc[5182]
o5000 = inf_alloys.loc[5052]  + inf_alloys.loc['other 5000 series']
a6063 = inf_alloys.loc[6063]
a6000 = inf_alloys.loc['Other 6000 series'] + inf_alloys.loc[6061]
s7000 = inf_alloys.loc['7000 series'] 
s8000 = inf_alloys.loc['8000 series'] 
cast_al = inf_alloys.loc['Cast'] + inf_alloys.loc['Die-Cast']

labels = ['1000 series','2000 series','3004 alloy','Other 3000 series','5182 alloys','Other 5000 series','6063 alloy','6000 series','7000 series','8000 series','Cast alloys']
ax2.stackplot(x,  s1000, s2000, a3004, o3000, a5182, o5000, a6063, a6000, s7000, s8000, cast_al, colors = colors, labels=labels)
ax2.yaxis.set_minor_locator(plt.MultipleLocator(6.25))
ax2.set_ylim(ymax=140)
ax2.set(ylabel='Inflow [Mt]')


# ax3 - Plot inflow

s1000 = out_alloys.loc['1000 series']
s2000 = out_alloys.loc['2000 series']
a3004 = out_alloys.loc[3004]
o3000 = out_alloys.loc[3003] + out_alloys.loc['other 3000 series']
a5182 = out_alloys.loc[5182]
o5000 = out_alloys.loc[5052]  + out_alloys.loc['other 5000 series']
a6063 = out_alloys.loc[6063]
a6000 = out_alloys.loc['Other 6000 series'] + out_alloys.loc[6061]
s7000 = out_alloys.loc['7000 series'] 
s8000 = out_alloys.loc['8000 series'] 
cast_al = out_alloys.loc['Cast'] + out_alloys.loc['Die-Cast']

labels = ['1000 series','2000 series','3004 alloy','Other 3000 series','5182 alloy','Other 5000 series','6063 alloy','6000 series','7000 series','8000 series','Cast alloys']
ax3.stackplot(x,  s1000, s2000, a3004, o3000, a5182, o5000, a6063, a6000, s7000, s8000, cast_al, colors = colors, labels=labels)
ax3.yaxis.set_minor_locator(plt.MultipleLocator(6.25))
ax3.set_ylim(ymax=140)
ax3.set(ylabel='Outflow [Mt]')
    
#Remove axis
for ax in fig.get_axes():
    ax.label_outer()
    ax.xaxis.set_minor_locator(plt.MultipleLocator(5))
    #Start y axis by 0
    ax.set_ylim(ymin=0)
    

#Legends
legend_stages = plt.legend(reversed(plt.legend().legendHandles), reversed(labels),
                       title = 'Alloys', loc=(1.02, 0.8))
legend = plt.gca().add_artist(legend_stages)    

# x label
plt.xlabel('year')
plt.xticks(list(range(2020,2101,10)))

#Remove axis
for ax in fig.get_axes():
    ax.label_outer()
    #Start y axis by 0
    ax.set_ylim(ymin=0)
    ax.set_xlim(xmin = 2013 , xmax=2102)
    
plt.savefig('figure5.pdf', dpi = 900, bbox_inches='tight')

# Figure 6 - Sensitivity Aluminium content

In [ ]:
#sector = 
stock_ssp = stock.loc[:,:,:,:].sum(level=[0,2])
inflow_ssp = inflow.loc[:,:,:,:].sum(level=[0,2])
outflow_ssp = outflow.loc[:,:,:,:].sum(level=[0,2])
SSP  = list(dict.fromkeys(stock_ssp.index.get_level_values(0)))
Level  = list(dict.fromkeys(stock_ssp.index.get_level_values(1)))

In [ ]:
x = stock.columns

fig, (ax1, ax2, ax3) = plt.subplots(3)
fig.set_figheight(12)
fig.set_figwidth(10)
fig.subplots_adjust(hspace=0.1)
colors = ['green','blue','red','orange','purple']
SSP = ['SSP1','SSP2','SSP3','SSP4','SSP5']

#Plot stock
for ssp in np.arange(0,len(SSP)):
    y_med = stock_ssp.loc[SSP[ssp]].loc['Medium']
    ax1.plot(x, y_med, c=colors[ssp], lw=3)
    y_low = stock_ssp.loc[SSP[ssp]].loc['Low']
    ax1.plot(x, y_low, c=colors[ssp], lw=1, ls=':')
    y_high = stock_ssp.loc[SSP[ssp]].loc['High']
    ax1.plot(x, y_high, c=colors[ssp], lw=1, ls='-.')
    ax1.fill_between(x, y_low, y_high, facecolor=colors[ssp], alpha=0.05, edgecolor=colors[ssp]) 
    plt.ylabel('Stock')
    
ax1.yaxis.set_minor_locator(plt.MultipleLocator(250))
ax1.set(ylabel='Stock [unit]')
    
#Plot inflow
for ssp in np.arange(0,len(SSP)):
    y_med = inflow_ssp.loc[SSP[ssp]].loc['Medium']
    ax2.plot(x, y_med, c=colors[ssp], lw=3)
    y_low = inflow_ssp.loc[SSP[ssp]].loc['Low']
    ax2.plot(x, y_low, c=colors[ssp], lw=1, ls=':')
    y_high = inflow_ssp.loc[SSP[ssp]].loc['High']
    ax2.plot(x, y_high, c=colors[ssp], lw=1, ls='-.')
    ax2.fill_between(x, y_low, y_high, facecolor=colors[ssp], alpha=0.05, edgecolor=colors[ssp])
ax2.yaxis.set_minor_locator(plt.MultipleLocator(12.5))    
ax2.set(ylabel='Inflow [unit]')

 
#Plot outflow
for ssp in np.arange(0,len(SSP)):
    y_med = outflow_ssp.loc[SSP[ssp]].loc['Medium']
    ax3.plot(x, y_med, c=colors[ssp], lw=3)
    y_low = outflow_ssp.loc[SSP[ssp]].loc['Low']
    ax3.plot(x, y_low, c=colors[ssp], lw=1, ls=':')
    y_high = outflow_ssp.loc[SSP[ssp]].loc['High']
    ax3.plot(x, y_high, c=colors[ssp], lw=1, ls='-.')
    ax3.fill_between(x, y_low, y_high, facecolor=colors[ssp], alpha=0.05, edgecolor=colors[ssp])
ax3.yaxis.set_minor_locator(plt.MultipleLocator(12.5))    
ax3.set(ylabel='Outflow [unit]')
    
#Remove axis
for ax in fig.get_axes():
    ax.label_outer()
    ax.xaxis.set_minor_locator(plt.MultipleLocator(5))
    #Start y axis by 0
    ax.set_ylim(ymin=0)
    
ax2.set_ylim(ymax=280)
ax3.set_ylim(ymax=280)

plt.xticks(list(range(2020,2101,10)))
#Add legend
labels = ['SSP1 - Medium','SSP1 - Low', 'SSP1 - High', 'SSP2 - Medium','SSP2 - Low', 'SSP2 - High',
         'SSP3 - Medium','SSP3 - Low', 'SSP3 - High','SSP4 - Medium','SSP4 - Low', 'SSP4 - High',
         'SSP5 - Medium','SSP5 - Low', 'SSP5 - High']
plt.legend(labels, fontsize=12, loc=((1.03, 0.87)))

plt.xlabel('year')
plt.savefig('figure6.pdf', dpi = 900, bbox_inches='tight')


# Supplementary information

## SI-Figure 1 - Stock - SSP - Regions

In [ ]:
#Total stock
df = dMFA.stock.loc[:,:,'Medium'].sum(level=[0,1])
subplot_SSP_region(df,'[Mt]','Total stock',50)

## SI-Figure 2 - Inflow - SSP - Regions

In [ ]:
#Inflow
df = dMFA.inflow.loc[:,:,'Medium'].sum(level=[0,1])
subplot_SSP_region(df,'[Mt]','Total inflow',2.5)

## SI-Figure 3 - Outflow - SSP - Regions

In [ ]:
#Outflow
df = dMFA.outflow.loc[:,:,'Medium'].sum(level=[0,1])
subplot_SSP_region(df,'[Mt]','Total outflow',2.5)

## SI-Figure 4 - Sensitivity analysis - Lifetime

In [ ]:
dMFA_short_lifetime = Prastof.from_template(data_file)
dMFA_short_lifetime.lifetime = dMFA_short_lifetime.lifetime*0.75
dMFA_short_lifetime.solve()
inflow_s = dMFA_short_lifetime.inflow 
outflow_s = dMFA_short_lifetime.outflow 
stock_s = dMFA_short_lifetime.stock
stock_cap_s = dMFA_short_lifetime.stock_cap
inflow_s[2015] = inflow_s[2016]

In [ ]:
dMFA_long_lifetime = Prastof.from_template(data_file)
dMFA_long_lifetime.lifetime = dMFA.lifetime*1.25
dMFA_long_lifetime.solve()

inflow_l = dMFA_long_lifetime.inflow 
outflow_l = dMFA_long_lifetime.outflow 
stock_l = dMFA_long_lifetime.stock
stock_cap_l = dMFA_long_lifetime.stock_cap 
inflow_l[2015] = inflow_l[2016]

In [ ]:
#sector
level = 'Medium'

inflow_ssp = inflow.loc[:,:,level,:].sum(level=[0])
inflow_ssp_s = inflow_s.loc[:,:,level,:].sum(level=[0])
inflow_ssp_l = inflow_l.loc[:,:,level,:].sum(level=[0])

outflow_ssp = outflow.loc[:,:,level,:].sum(level=[0])
outflow_ssp_s = outflow_s.loc[:,:,level,:].sum(level=[0])
outflow_ssp_l = outflow_l.loc[:,:,level,:].sum(level=[0])

SSP  = list(dict.fromkeys(stock_ssp.index.get_level_values(0)))

In [ ]:
x = stock.columns

fig, (ax1, ax2) = plt.subplots(2)

fig.set_figwidth(10)
fig.set_figheight(8)

fig.subplots_adjust(hspace=0.1)
colors = ['green','blue','red','orange','purple']
SSP = ['SSP1','SSP2','SSP3','SSP4','SSP5']
    
#Plot inflow
for ssp in np.arange(0,len(SSP)):
    y_med = inflow_ssp.loc[SSP[ssp]]
    ax1.plot(x, y_med, c=colors[ssp], lw=3)
    y_short = inflow_ssp_s.loc[SSP[ssp]]
    ax1.plot(x, y_short, c=colors[ssp], lw=1, ls=':')
    y_long = inflow_ssp_l.loc[SSP[ssp]]
    ax1.plot(x, y_long, c=colors[ssp], lw=1, ls='-.')
    ax1.fill_between(x, y_short, y_long, facecolor=colors[ssp], alpha=0.05, edgecolor=colors[ssp])
ax1.yaxis.set_minor_locator(plt.MultipleLocator(12.5))    
ax1.set(ylabel='Inflow [unit]')

 
#Plot outflow
for ssp in np.arange(0,len(SSP)):
    y_med = outflow_ssp.loc[SSP[ssp]]
    ax2.plot(x, y_med, c=colors[ssp], lw=3)
    y_short = outflow_ssp_s.loc[SSP[ssp]]
    ax2.plot(x, y_short, c=colors[ssp], lw=1, ls=':')
    y_long = outflow_ssp_l.loc[SSP[ssp]]
    ax2.plot(x, y_long, c=colors[ssp], lw=1, ls='-.')
    ax2.fill_between(x, y_short, y_long, facecolor=colors[ssp], alpha=0.05, edgecolor=colors[ssp])

ax2.yaxis.set_minor_locator(plt.MultipleLocator(12.5))
ax2.set(ylabel='Outflow [unit]')
    
#Remove axis
for ax in fig.get_axes():
    ax.label_outer()
    ax.xaxis.set_minor_locator(plt.MultipleLocator(5))
    #Start y axis by 0
    ax.set_ylim(ymin=0)
    
ax1.set_ylim(ymax=250)
ax2.set_ylim(ymax=250)
plt.xticks(list(range(2020,2101,10)))

#Add legend
labels = ['SSP1','SSP1 - Short', 'SSP1 - Long', 'SSP2','SSP2 - Short', 'SSP2 - Long',
         'SSP3','SSP3 - Short', 'SSP3 - Long','SSP4','SSP4 - Short', 'SSP4 - Long',
         'SSP5','SSP5 - Short', 'SSP5 - Long']
plt.legend(reversed(plt.legend().legendHandles), reversed(labels), loc=(1.03, 0.37))
plt.xlabel('year')


## SI-Figure 5 - Comparison literature

In [ ]:
#Data from alu cycle
data_alu_cycle  = pd.DataFrame([[122,147,168,190,213,237,261,285],
                                [900,1145,1375,1625,1893,2181,2483,2785],
                                [70,77.5,88.8,102.6,115,129.6,140.7,150],
                                [22.6,26,37.2,44.2,52.8,61.8,71.5,80.7]], 
                               index=['Stock/capita','Stock','Inflow','Outflow'],
                              columns=[2015,2020,2025,2030,2035,2040,2045,2050])

In [ ]:
#Data from (Muller et al., 2013)
#Stock per capita
data_muller = pd.DataFrame([[ 105, 120, 180, 220, 310, 400, 500, 600, 600, 600, 600, 600, 600],
                            [ 105, 110, 150, 180, 210, 250, 300, 360, 500 , 550, 600, 600, 600],
                            [ 105, 110, 140, 160, 180, 210, 250, 285, 360, 420, 490, 550, 600],
                            [ 105, 110, 140, 190, 230, 300, 350, 400, 400, 400, 400, 400, 400],
                            [ 105, 110, 140, 160, 180, 210, 250, 285, 315, 360, 400, 400, 400],
                            [ 105, 110, 130, 155, 170, 190, 210, 245, 280, 310, 340, 360, 400],
                            [ 105, 110, 130, 150, 160, 175, 190, 200, 200, 200, 200, 200, 200],
                            [ 105, 110, 120, 140, 150, 160,172.5,180, 185,187.5, 190, 200, 200],
                            [ 105, 110, 120, 140, 150, 160, 170, 175, 180, 185, 190, 195, 200]], 
                            index=['High-2050','High-2075','High-2100', 'Medium-2050','Medium-2075', 'Medium-2100', 'Low-2050','Low-2075', 'Low-2100'],
                    columns=[2015,2020,2025,2030,2035,2040,2045,2050,2060,2070,2080,2090,2100])

In [ ]:
#Data from (Watari et al., 2021)
#inflow
data_Watari  = pd.DataFrame([[60, 120,180,330]], index=['Inflow'],
                     columns=[2015,2030,2050,2100])

In [ ]:
# Stock per capita
stock_cap_ssp_level = stock_cap.loc[:,'GLO',:].sum(level=[0,1])

In [ ]:
fig, (ax1) = plt.subplots(1)

fig.set_figwidth(8)
fig.set_figheight(6)

colors = ['green','blue','red','orange','purple']
SSP = ['SSP1','SSP2','SSP3','SSP4','SSP5']
    
#Plot inflow
for ssp in np.arange(0,len(SSP)):
    y_med = stock_cap_ssp_level.loc[SSP[ssp]].loc['Medium']
    ax1.plot(x, y_med, c=colors[ssp], lw=3, ls='-')
    #y_low = stock_cap_ssp_level.loc[SSP[ssp]].loc['Low']
    #ax1.plot(x, y_low, c=colors[ssp], lw=2, ls='--')
    #y_high = stock_cap_ssp_level.loc[SSP[ssp]].loc['High']
    #ax1.plot(x, y_high, c=colors[ssp], lw=2, ls=':')
    
y_mul_h_2100 = data_muller.loc['High-2100']
ax1.plot(y_mul_h_2100, color='grey', alpha=1, lw=2)
y_mul_h_2100 = data_muller.loc['High-2075']
ax1.plot(y_mul_h_2100, color='grey', alpha=0.75, lw=2)
y_mul_h_2100 = data_muller.loc['High-2050']
ax1.plot(y_mul_h_2100, color='grey', alpha=0.5, lw=2)

y_mul_h_2100 = data_muller.loc['Medium-2100']
ax1.plot(y_mul_h_2100, color='grey', alpha=1, ls=':', lw=2)
y_mul_h_2100 = data_muller.loc['Medium-2075']
ax1.plot(y_mul_h_2100, color='grey', alpha=0.75, ls=':', lw=2)
y_mul_h_2100 = data_muller.loc['Medium-2050']
ax1.plot(y_mul_h_2100, color='grey', alpha=0.5, ls=':', lw=2)

y_mul_h_2100 = data_muller.loc['Low-2100']
ax1.plot(y_mul_h_2100, color='grey', alpha=1, ls='--', lw=2)
y_mul_h_2100 = data_muller.loc['Low-2075']
ax1.plot(y_mul_h_2100, color='grey', alpha=0.75, ls='--', lw=2)
y_mul_h_2100 = data_muller.loc['Low-2050']
ax1.plot(y_mul_h_2100, color='grey', alpha=0.5, ls='--', lw=2)


ax1.plot(data_alu_cycle.loc['Stock/capita'], color='k',lw=4, ls='-')
plt.legend(['SSP1', 'SSP2', 'SSP3','SSP4','SSP5','High-2100', 'High-2075', 'High-2050', 
            'Medium-2100','Medium-2075','Medium-2050', 'Low-2100','Low-2075','Low-2050','Alu-Cycle'], title='Scenarios', loc=((1.04, 0.15)))

ax1.yaxis.set_minor_locator(plt.MultipleLocator(25))
ax1.set_ylim(ymin=0)
ax1.set(ylabel='Stock [kg/cap]')

ax1.set(xlabel='Year')

In [ ]:
# Stock
stock_ssp_level = dMFA.stock.loc[:,:,:].sum(level=[0,2])

In [ ]:
fig, (ax1) = plt.subplots(1)

fig.set_figwidth(8)
fig.set_figheight(6)

colors = ['green','blue','red','orange','purple']
SSP = ['SSP1','SSP2','SSP3','SSP4','SSP5']
    
#Plot inflow
for ssp in np.arange(0,len(SSP)):
    y_med = stock_ssp_level.loc[SSP[ssp]].loc['Medium']
    ax1.plot(x, y_med, c=colors[ssp], lw=2, ls='-')
    y_low = stock_ssp_level.loc[SSP[ssp]].loc['Low']
    ax1.plot(x, y_low, c=colors[ssp], lw=2, ls='--')
    y_high = stock_ssp_level.loc[SSP[ssp]].loc['High']
    ax1.plot(x, y_high, c=colors[ssp], lw=2, ls=':')
    
ax1.plot(data_alu_cycle.loc['Stock'], color='k',lw=4, ls='-')

plt.legend(['SSP1-Medium','SSP1-Low','SSP1-High', 
            'SSP2-Medium','SSP2-Low','SSP2-High',
            'SSP3-Medium','SSP3-Low','SSP3-High',
            'SSP4-Medium','SSP4-Low','SSP4-High',
            'SSP5-Medium','SSP5-Low','SSP5-High',
            'Alu-Cycle'], title='Scenarios', loc=((1.04, 0.15)))

ax1.yaxis.set_minor_locator(plt.MultipleLocator(250))
ax1.set_ylim(ymin=0)
ax1.set(ylabel='Stock [Mt]')
ax1.set(xlabel='Year')

In [ ]:
# Inflow
inf_ssp_level = dMFA.inflow.loc[:,:,:].sum(level=[0,2])

In [ ]:
fig, (ax1) = plt.subplots(1)

fig.set_figwidth(8)
fig.set_figheight(6)

colors = ['green','blue','red','orange','purple']
SSP = ['SSP1','SSP2','SSP3','SSP4','SSP5']
    
#Plot inflow
for ssp in np.arange(0,len(SSP)):
    y_med = inf_ssp_level.loc[SSP[ssp]].loc['Medium']
    ax1.plot(x, y_med, c=colors[ssp], lw=2, ls='-')
    y_low = inf_ssp_level.loc[SSP[ssp]].loc['Low']
    ax1.plot(x, y_low, c=colors[ssp], lw=2, ls='--')
    y_high = inf_ssp_level.loc[SSP[ssp]].loc['High']
    ax1.plot(x, y_high, c=colors[ssp], lw=2, ls=':')
    
ax1.plot(data_alu_cycle.loc['Inflow'], color='k',lw=4, ls='-')

ax1.plot(data_Watari.loc['Inflow'], color='grey',lw=3, ls='-')
    
plt.legend(['SSP1-Medium','SSP1-Low','SSP1-High', 
            'SSP2-Medium','SSP2-Low','SSP2-High',
            'SSP3-Medium','SSP3-Low','SSP3-High',
            'SSP4-Medium','SSP4-Low','SSP4-High',
            'SSP5-Medium','SSP5-Low','SSP5-High',
            'Alu-Cycle', 'Review-Watari et al.'], title='Scenarios', loc=((1.04, 0.15)))

ax1.yaxis.set_minor_locator(plt.MultipleLocator(12.5))
ax1.set_ylim(ymin=0)
ax1.set(ylabel='Inflow [Mt]')
ax1.set(xlabel='Year')

In [ ]:
# Outflow
out_ssp_level = dMFA.outflow.loc[:,:,:].sum(level=[0,2])

In [ ]:
fig, (ax1) = plt.subplots(1)

fig.set_figwidth(8)
fig.set_figheight(6)

colors = ['green','blue','red','orange','purple']
SSP = ['SSP1','SSP2','SSP3','SSP4','SSP5']
    
#Plot inflow
for ssp in np.arange(0,len(SSP)):
    y_med = out_ssp_level.loc[SSP[ssp]].loc['Medium']
    ax1.plot(x, y_med, c=colors[ssp], lw=2, ls='-')
    y_low = out_ssp_level.loc[SSP[ssp]].loc['Low']
    ax1.plot(x, y_low, c=colors[ssp], lw=2, ls='--')
    y_high = out_ssp_level.loc[SSP[ssp]].loc['High']
    ax1.plot(x, y_high, c=colors[ssp], lw=2, ls=':')
    
ax1.plot(data_alu_cycle.loc['Outflow'], color='k',lw=4, ls='-')
    
plt.legend(['SSP1-Medium','SSP1-Low','SSP1-High', 
            'SSP2-Medium','SSP2-Low','SSP2-High',
            'SSP3-Medium','SSP3-Low','SSP3-High',
            'SSP4-Medium','SSP4-Low','SSP4-High',
            'SSP5-Medium','SSP5-Low','SSP5-High',
            'Alu-Cycle'], title='Scenarios', loc=((1.04, 0.15)))

ax1.yaxis.set_minor_locator(plt.MultipleLocator(6.25))
ax1.set_ylim(ymin=0)
ax1.set(ylabel='Outflow [Mt]')
ax1.set(xlabel='Year')

# Export results

In [ ]:
#Create df from 2015 to 2100 with 5 years interval
stock_cap_to_export = stock_cap.loc(axis=1)[2015,2020,2025,2030,2035,2040,2045,2050,2055,2060,2065,2070,2075,2080,2085,2090,2095,2100] 
stock_to_export =  stock.loc(axis=1)[2015,2020,2025,2030,2035,2040,2045,2050,2055,2060,2065,2070,2075,2080,2085,2090,2095,2100] 
inflow_to_export =  inflow.loc(axis=1)[2015,2020,2025,2030,2035,2040,2045,2050,2055,2060,2065,2070,2075,2080,2085,2090,2095,2100] 
outflow_to_export = outflow.loc(axis=1)[2015,2020,2025,2030,2035,2040,2045,2050,2055,2060,2065,2070,2075,2080,2085,2090,2095,2100] 

In [ ]:
#Export results on excel

writer = pd.ExcelWriter('PRASTOF_results.xlsx', engine='xlsxwriter')

stock_cap_to_export.to_excel(writer, sheet_name='Stock_cap')
stock_to_export.to_excel(writer, sheet_name='Stock')
inflow_to_export.to_excel(writer, sheet_name='Inflow')
outflow_to_export.to_excel(writer, sheet_name='Outflow')

writer.save()